In [29]:
import pandas as pd
import numpy as np
import ast
import time

In [2]:
df = pd.read_csv("D:/DOWNLOADS/finalDataset.csv")
df.head()

,userID,movieID,rating,title,imdbID,keywords
0,8624,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
1,8930,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
2,12028,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
3,13916,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
4,15264,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."


In [3]:
n_data = pd.read_csv("D:/DOWNLOADS/movieNeighbors.csv")
n_data.head()

,Unnamed: 0,keys,values
0,0,Grumpy Old Men,"['Kind Hearts and Coronets', 'Song of the Sout..."
1,1,Twelve Monkeys,"['Breach', 'Outbreak', 'Meet the Robinsons', '..."
2,2,Braveheart,"['Local Hero', 'The Water Horse', 'Beverly Hil..."
3,3,Congo,"['Elektra', 'The Hand That Rocks the Cradle', ..."
4,4,Desperado,"['The Matrix Revolutions', 'Saw III', 'Sånger ..."


In [39]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def wscore(i, S, L, R):
    eta = 0.5
    overlap_S = woverlap(i, S, R)
    overlap_L_minus_S = woverlap(i, set(L) - set(S), R)
    return (1 - eta) * overlap_S + eta * overlap_L_minus_S

def woverlap(i, X, R):
    X = set(X)
    covered = set(item for sublist in [Ni(j) for j in R] for item in sublist)
    numerator = 2 * len((Ni(i) - covered).intersection(X))
    num=0
    for item in (Ni(i) - covered).intersection(X):
        try:
            j = I.index(item)
            num=num+w[j]
        except:
            num=0
    denominator = len(Ni(i) - covered) + len(X - covered)
    return 2*num / denominator if denominator > 0 else 0

def update(s, R, pi = "Open"):
    if(pi=="Open"):
        S = Ni(s)
        return S
    else:
        return {}
    
def argmax(Candidates, S, L, R):
    d=[]
    for i in Candidates:
        d.append([i,wscore(i, S, L, R)])
    d = sorted(d, key = lambda x: int(x[1]))
    i_star = d[0][0]
    return i_star

In [16]:
user_profile = df[df['userID']==8624]
keywords = df['keywords'].unique()
movies = df['title'].unique()
print(user_profile.shape)
seen = user_profile['title'].tolist()
L=[]
D=dict()
c=0
for i in user_profile.values:
    c=c+1
    #print("COUNT:",c)
    li=[]
    mi = i[3]
    try:
        tag = i[-1].split(',')
        tag = set(tag)
        for k in keywords:
            try:
                tm = set(k.split(','))
                sim = jaccard_similarity(tag, tm)
                #print(tag, tm)
                if(sim>0):
                    li.append(k)
                    L.append(k)
            except:
                #print("No tag found")
                continue
        D[mi] = li
    except:
        continue
         
N=dict()
L=[]
for i in D:
    li=[]
    for k in D[i]:
        mov = df[df['keywords']==k]['title'].tolist()
        mov = list(set(mov))
        for m in mov:
            if(m!=i and m not in seen):
                li.append(m)
                L.append(m)
    N[i] = li
L = list(set(L))   

(95, 6)


In [17]:
up = user_profile['title'].tolist()
up[:10]

['Grumpy Old Men',
 'La cité des enfants perdus',
 'The Shawshank Redemption',
 'Four Weddings and a Funeral',
 'Mystery Science Theater 3000: The Movie',
 'Wallace and Gromit in A Close Shave',
 'A Fish Called Wanda',
 'Life of Brian',
 'Monty Python and the Holy Grail',
 'Wallace & Gromit in The Wrong Trousers']

In [18]:
I = list(set(movies) - set(up))
len(I)

6386

In [19]:
len(movies)

6479

In [20]:
def Nj(i):
    ke=[]
    try:
        tag = set(df[df['title']==i]['keywords'].tolist()[0].split(','))
    except:
        return set([])
    for k in keywords:
        try:
            tm = set(k.split(','))
            sim = jaccard_similarity(tag, tm)
            if(sim>0):
                ke.append(k)
        except:
            continue
    ke = list(set(ke))
    li = []
    for k in ke:
        m = df[df['keywords']==k]['title'].tolist()
        li = li + m 
    return set(li)
        
def Ni(i):
    li = n_data[n_data['keys']==i]['values'].tolist()[0]
    li = ast.literal_eval(li)
    return set(li)

In [33]:
def Reweigh(s, R, rho="Direc"):
    
    for i in range(len(I)):
        Cij = 0
        for j in R:
            Cij = Cij + C(i,j)
        dwi = C(i,s) - Cij
        w[i] = w[i] + dwi
def C(i,j):
    if(i in Ni(j)):
        return 1
    else:
        return 0

In [34]:
w=[]
for i in range(len(I)):
    w.append(0)
def nbycp(s, L, rho = "Direc", n=5):
    S = N[s]
    Tabu = set([])
    Reweigh(s, set([]), rho)
    while(len(S)>n):
        R = recommend(S, L, n)
        print(R)
        time.sleep(2)
        print("Liked Item: ")
        s = input()
        print("Stop/Continue: ")
        a = input()
        if(a=="Stop"):
            return s
        R = R - set(s)
        S = update(s, R, pi="Open")
        Reweigh(s, R, rho)
        Tabu = Tabu.union(R)
        S = S - Tabu

In [35]:
def recommend(S, L, n=2):
    Candidates = set(S)
    R = set([])
    while len(R) < n and len(Candidates) > 0:
        #print(R)
        i_star = argmax(Candidates, S, L, R)
        #print(i_star)
        R.add(i_star)
        Candidates.remove(i_star)
        #print(Candidates)
    return R

In [40]:
nbycp("Grumpy Old Men", L, n=5)

{'Nothing But Trouble', 'Richard III', 'Key Largo', 'Sgt. Bilko', "Widows' Peak"}
Liked Item: 
Richard III
Stop/Continue: 
Continue
{'Henry V', 'Les triplettes de Belleville', 'Finding Neverland', 'Balto', 'Slaughterhouse-Five'}
Liked Item: 
Henry V
Stop/Continue: 
Stop


'Henry V'